In [2]:
import recordlinkage
import pandas as pd
import pickle

In [3]:
config_path = '/home/edkl/project/20222/THDL/BTL/THDL/config_mapping'

In [4]:
with open(config_path + '/muaban_okxe.pkl', 'rb') as f:
    muaban_okxe = pickle.load(f)
with open(config_path + '/muaban_webike.pkl', 'rb') as f:
    muaban_webike = pickle.load(f)
with open(config_path + '/muaban_xemayhoangkien.pkl', 'rb') as f:
    muaban_xemayhoangkien = pickle.load(f)

In [5]:
xemayhoangkien_inv = {value: key for (key, value) in muaban_xemayhoangkien.items()}
okxe_inv = {value: key for (key, value) in muaban_okxe.items()}
webike_inv = {value: key for (key, value) in muaban_webike.items()}

In [6]:
xemayhoangkien_inv

{'price': 'price',
 'address': 'address',
 'name': 'name',
 'url': 'url',
 'Năm đăng ký': 'Năm đăng ký',
 'Màu sắc': 'Màu xe',
 'Dung tích': 'Dung tích xe máy',
 'Thương hiệu': 'Hãng xe',
 'Loại': 'Loại xe máy'}

In [7]:
data_folder = "../crawl/crawl/spiders/all_data"
okxe_data = pd.read_csv(data_folder + "/okxe.csv").drop(columns=['Unnamed: 0'], axis=1)
muaban_data = pd.read_csv(data_folder + "/muaban.csv").drop(columns=['Unnamed: 0'], axis=1)
webike_data = pd.read_csv(data_folder + "/webike.csv").drop(columns=['Unnamed: 0'], axis=1)
xemayhoangkien_data = pd.read_csv(data_folder + "/xemayhoangkien.csv").drop(columns=['Unnamed: 0'], axis=1)

In [8]:
xemayhoangkien_data.head()

,name,price,url,Thương hiệu,Loại,MSP,Năm đăng ký,Dung tích,Màu sắc,address
0,Winner X ABS 22Y-222BG,39500000,https://xemayhoangkien.com/winner-x-abs-29y-222bg,Honda,Winner,-,-,-,-,52 Chùa Hà - Cầu Giấy - Hà Nội
1,LX 30L-48DC,14200000,https://xemayhoangkien.com/lx-30l-48dc,Piaggio,LX,-,-,-,-,52 Chùa Hà - Cầu Giấy - Hà Nội
2,Wave A 38K-545CB,15800000,https://xemayhoangkien.com/wave-a-38k-545cb,Honda,Wave A,38K-545CB,2020,110cc,Trắng đen bạc,52 Chùa Hà - Cầu Giấy - Hà Nội
3,LX ie 29X-113JK,20000000,https://xemayhoangkien.com/lx-ie-29x-113jk,Piaggio,LX,29X-113JK,2013 (đời 2012),125cc,Vàng,52 Chùa Hà - Cầu Giấy - Hà Nội
4,TFX 89B-465GG,39300000,https://xemayhoangkien.com/tfx-89b-465gg,Yamaha,TFX,89B-465GG,2017,150cc,Đỏ đen,52 Chùa Hà - Cầu Giấy - Hà Nội


In [9]:
muaban_data.head()

,name,price,address,url,Hãng xe,Năm đăng ký,Tình trạng,Loại xe máy,Dung tích xe máy,Xuất xứ,Màu xe,Dòng xe,Số km đã đi
0,CẦN BÁN XE MÁY HONDA VISION 110 ĐỜI 2022,15000000,"Quận 5, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-ban-x...,Honda,2022.0,Mới,Tay ga,101 - 175 cc,Nước khác,Đen,-,0
1,"Chuyên thu mua các loại xe máy cũ và mới, xe h...",0,"Quận Đống Đa, Hà Nội",https://muaban.net/xe-may/mua-xe-may/chuyen-th...,-,-,-,-,-,-,-,-,0
2,HEAD SƠN TÙNG 1 - TRỢ GIÁ MUA XE TƯNG BỪNG-VUI...,31990000,"Huyện Hoài Đức, Hà Nội",https://muaban.net/xe-may/ban-xe-may/head-son-...,Honda,-,Mới,Tay côn/Moto,-,-,-,-,0
3,Cần thanh lý lô xe honda AirBlade 150 nhập khẩu,19000000,"TP. Thủ Đức - Quận 9, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-thanh...,Honda,2022.0,Mới,Tay ga,101 - 175 cc,Nước khác,Đen,Air Blade,0
4,CẦN BÁN XE MÁY HONDA SH150 ĐỜI 2022,32000000,"Quận 5, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-ban-x...,Honda,2023.0,Mới,Tay ga,101 - 175 cc,Nước khác,Ghi/Xám,SH,0


In [10]:
webike_data.head()

,name,price,address,url,Dòng xe,Đã đi,Xuất xứ,Đời xe,Phân khối,Loại xe,Màu,Ngày đăng
0,Cần bán PIAGGIO Liberty 3V i.e 125 2015 màu trắng,19500000.0,"Cầu Giấy , Hà Nội",https://www.webike.vn/cho-xe-may/bike-detail/c...,PIAGGIO Liberty 3V i.e 125 2015,10 000 - 14 999 km,Việt Nam,2015,125 cc,Xe tay ga,Trắng,11 ngày trước
1,HONDA Airblade 110 Fi Đen Đỏ Trắng - 2013,10500000.0,"Cầu Giấy , Hà Nội",https://www.webike.vn/cho-xe-may/bike-detail/h...,HONDA Airblade 110 Fi 2013,5 000 - 9 999 km,Nhật Bản,2013,110 cc,Xe tay ga,Đen Đỏ Trắng,16 ngày trước
2,Cần bán HONDA SH 150i 2014 màu trắng,60000000.0,"An Khê , Gia Lai",https://www.webike.vn/cho-xe-may/bike-detail/c...,HONDA SH 150i 2014,40 000 - 44 999 km,Việt Nam,2014,150 cc,Xe tay ga,Trắng,5 ngày trước
3,Cần bán PIAGGIO Vespa S 3V i.e 125 2016 màu xa...,0.0,"Cầu Giấy , Hà Nội",https://www.webike.vn/cho-xe-may/bike-detail/c...,PIAGGIO Vespa S 3V i.e 125 2016,10 000 - 14 999 km,Việt Nam,2016,125 cc,Xe tay ga,Xanh Da Trời Xám,1 tháng trước
4,HONDA SH Mode 125 Đen Đỏ - 2019,47000000.0,"Tân Phú , Hồ Chí Minh",https://www.webike.vn/cho-xe-may/bike-detail/h...,HONDA SH Mode 125 2019,15 000 - 19 999 km,Việt Nam,2019,125 cc,Xe tay ga,Đen Đỏ,16 ngày trước


In [11]:
okxe_data.head()

,name,price,address,url,brand,image_url,model,store,type,fuel,...,used_distance,Giảm xóc - Phuộc,Phanh,Động cơ,Đồng hồ,Khoá,Hệ thống đèn,Hệ thống tự động,Công nghệ kết nối,Khác
0,Suzuki GSX 150 Bandit,28000000,Tp. Hồ Chí Minh,https://okxe.vn/products/suzuki-gsx-150-brandi...,Suzuki,https://cdn.okxe.vn/products/848/1689848/16898...,GSX,Trạm Dịch Vụ Xe Máy TP HCM,2,Gasolin,...,0-10600km,Phuộc Showa SFF-CA & monoshock Showa BFRC,Phanh Đĩa,Phun xăng điện tử Fi / PGM-FI,Đồng hồ LCD,-,-,-,-,-
1,Honda Air Blade 160 FI,58000000,Tp. Hà Nội,https://okxe.vn/products/honda-air-blade-160-f...,Honda,https://cdn.okxe.vn/products/503/1690503/16905...,Air Blade,Trạm Dịch Vụ Xe Máy OKXE Hà Nội,1,Gasolin,...,0-3000km,-,"Phanh ABS,Phanh Đĩa",Phun xăng điện tử Fi / PGM-FI,Đồng hồ điện tử,"Smartkey,Khóa chống trộm",Đèn LED,Hệ thống ngắt động cơ tạm thời - Idling Stop,USB post,"Tiết kiệm nhiên liệu,Hiển thị Eco"
2,Honda Air Blade 125 FI,35000000,Tp. Hồ Chí Minh,https://okxe.vn/products/honda-air-blade-125-f...,Honda,https://cdn.okxe.vn/products/241/1683241/16832...,Air Blade,Trạm Dịch Vụ Xe Máy TP HCM,1,Gasolin,...,0-7000km,-,"Phanh cơ,Phanh Đĩa",Phun xăng điện tử Fi / PGM-FI,-,-,-,Hệ thống ngắt động cơ tạm thời - Idling Stop,-,-
3,Honda Air Blade 125 FI,30500000,Tp. Hồ Chí Minh,https://okxe.vn/products/honda-air-blade-125-f...,Honda,https://cdn.okxe.vn/products/822/1689822/16898...,Air Blade,Trạm Dịch Vụ Xe Máy TP HCM,1,Gasolin,...,0-60000km,-,"Phanh cơ,Phanh Đĩa",Phun xăng điện tử Fi / PGM-FI,-,-,-,Hệ thống ngắt động cơ tạm thời - Idling Stop,-,-
4,Honda SH 125i,60000000,Tp. Hồ Chí Minh,https://okxe.vn/products/honda-sh-125i-1689740,Honda,https://cdn.okxe.vn/products/740/1689740/16897...,SH,Trạm Dịch Vụ Xe Máy TP HCM,1,Gasolin,...,0-5000km,-,-,Phun xăng điện tử Fi / PGM-FI,Đồng hồ LCD,-,-,Hệ thống ngắt động cơ tạm thời - Idling Stop,-,-


In [12]:
all = pd.DataFrame(columns=['name', 'price', 'address', 'url', 'brand', 'registerYear', 'status', 'type', 'origin', 'capacity', 'color', 'vehicleType', 'km', 'imageUrl'])
all

,name,price,address,url,brand,registerYear,status,type,origin,capacity,color,vehicleType,km,imageUrl


In [13]:
# Build the indexer
indexer = recordlinkage.Index()
# Can use full or block
# indexer.full()
# indexer.block(left_on='State', right_on='Provider State')

# Use sortedneighbor as a good option if data is not clean
indexer.sortedneighbourhood(left_on='name', right_on='name')
candidates = indexer.index(muaban_data, webike_data)
compare = recordlinkage.Compare()
# compare.exact('City', 'Provider City', label='City')
compare.string('name','name', method='levenshtein',threshold=0.9,label='Name')
compare.string('Dòng xe', 'Dòng xe', method='jarowinkler', threshold=0.5, label='Dòng xe')
compare.string('address', 'address', method='jarowinkler', threshold=0.5, label='address')
compare.string('Loại xe máy', 'Loại xe', method='jarowinkler', threshold=0.5, label='Loại xe')
compare.exact('Số km đã đi', 'Đã đi', label='Đã đi')
features = compare.compute(candidates, muaban_data,
                           webike_data)
# Get the potential matches
potential_matches = features[features.sum(axis=1) > 3].reset_index()
potential_matches

,level_0,level_1,Name,Dòng xe,address,Loại xe,Đã đi
0,436,1545,1.0,1.0,1.0,1.0,0
1,187,1545,1.0,1.0,1.0,1.0,0


In [14]:
webike_data[~webike_data.index.isin(potential_matches['level_1'])]

,name,price,address,url,Dòng xe,Đã đi,Xuất xứ,Đời xe,Phân khối,Loại xe,Màu,Ngày đăng
0,Cần bán PIAGGIO Liberty 3V i.e 125 2015 màu trắng,19500000.0,"Cầu Giấy , Hà Nội",https://www.webike.vn/cho-xe-may/bike-detail/c...,PIAGGIO Liberty 3V i.e 125 2015,10 000 - 14 999 km,Việt Nam,2015,125 cc,Xe tay ga,Trắng,11 ngày trước
1,HONDA Airblade 110 Fi Đen Đỏ Trắng - 2013,10500000.0,"Cầu Giấy , Hà Nội",https://www.webike.vn/cho-xe-may/bike-detail/h...,HONDA Airblade 110 Fi 2013,5 000 - 9 999 km,Nhật Bản,2013,110 cc,Xe tay ga,Đen Đỏ Trắng,16 ngày trước
2,Cần bán HONDA SH 150i 2014 màu trắng,60000000.0,"An Khê , Gia Lai",https://www.webike.vn/cho-xe-may/bike-detail/c...,HONDA SH 150i 2014,40 000 - 44 999 km,Việt Nam,2014,150 cc,Xe tay ga,Trắng,5 ngày trước
3,Cần bán PIAGGIO Vespa S 3V i.e 125 2016 màu xa...,0.0,"Cầu Giấy , Hà Nội",https://www.webike.vn/cho-xe-may/bike-detail/c...,PIAGGIO Vespa S 3V i.e 125 2016,10 000 - 14 999 km,Việt Nam,2016,125 cc,Xe tay ga,Xanh Da Trời Xám,1 tháng trước
4,HONDA SH Mode 125 Đen Đỏ - 2019,47000000.0,"Tân Phú , Hồ Chí Minh",https://www.webike.vn/cho-xe-may/bike-detail/h...,HONDA SH Mode 125 2019,15 000 - 19 999 km,Việt Nam,2019,125 cc,Xe tay ga,Đen Đỏ,16 ngày trước
...,...,...,...,...,...,...,...,...,...,...,...,...
2995,Sprint S125 Iget Abs Màu Xanh Sport 2021,70000000.0,"Quận 5 , Hồ Chí Minh",https://www.webike.vn/cho-xe-may/bike-detail/s...,PIAGGIO khác,5 000 - 9 999 km,-,-,-,Khác,-,4 tháng trước
2996,Xe Dream 2000 - Cần bán HONDA khác,11500000.0,"Hóc Môn , Hồ Chí Minh",https://www.webike.vn/cho-xe-may/bike-detail/x...,HONDA khác,30 000 - 34 999 km,-,-,-,Khác,-,4 tháng trước
2997,raider đen nhám biển 71 chính chủ,25500000.0,"Bình Thạnh , Hồ Chí Minh",https://www.webike.vn/cho-xe-may/bike-detail/r...,SUZUKI Raider R150 2018,0 - 4 999 km,-,2018,150 cc,Xe côn tay,-,4 tháng trước
2998,Virago 125 HQCN up full Cafe Racer,19000000.0,-,https://www.webike.vn/cho-xe-may/bike-detail/v...,-,-,-,-,-,-,-,-


In [15]:
dict_field = {
    'name': 'name', 
    'price': 'price', 
    'address': 'address', 
    'url': 'url', 
    'brand': 'Hãng xe', 
    'registerYear': 'Năm đăng ký', 
    'status': 'Tình trạng', 
    'type': 'Loại xe máy', 
    'origin': 'Xuất xứ', 
    'capacity': 'Dung tích xe máy', 
    'color': 'Màu xe', 
    'vehicleType': 'Dòng xe', 
    'km': 'Số km đã đi', 
    'imageUrl': 'image_url'
}

In [16]:
inverse_dict_field = {value: key for (key, value) in dict_field.items()}

In [17]:
all = all.append(muaban_data.rename(columns=inverse_dict_field))

/tmp/ipykernel_63266/2683210262.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all = all.append(muaban_data.rename(columns=inverse_dict_field))


In [18]:
all

,name,price,address,url,brand,registerYear,status,type,origin,capacity,color,vehicleType,km,imageUrl
0,CẦN BÁN XE MÁY HONDA VISION 110 ĐỜI 2022,15000000,"Quận 5, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-ban-x...,Honda,2022.0,Mới,Tay ga,Nước khác,101 - 175 cc,Đen,-,0,NaN
1,"Chuyên thu mua các loại xe máy cũ và mới, xe h...",0,"Quận Đống Đa, Hà Nội",https://muaban.net/xe-may/mua-xe-may/chuyen-th...,-,-,-,-,-,-,-,-,0,NaN
2,HEAD SƠN TÙNG 1 - TRỢ GIÁ MUA XE TƯNG BỪNG-VUI...,31990000,"Huyện Hoài Đức, Hà Nội",https://muaban.net/xe-may/ban-xe-may/head-son-...,Honda,-,Mới,Tay côn/Moto,-,-,-,-,0,NaN
3,Cần thanh lý lô xe honda AirBlade 150 nhập khẩu,19000000,"TP. Thủ Đức - Quận 9, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-thanh...,Honda,2022.0,Mới,Tay ga,Nước khác,101 - 175 cc,Đen,Air Blade,0,NaN
4,CẦN BÁN XE MÁY HONDA SH150 ĐỜI 2022,32000000,"Quận 5, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-ban-x...,Honda,2023.0,Mới,Tay ga,Nước khác,101 - 175 cc,Ghi/Xám,SH,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441,Suzuki Saphia máy êm ru chạy ngon như sapcy,4500000,-,https://muaban.net/xe-may/ban-xe-may-quan-5-ho...,-,-,-,-,-,-,-,-,0,NaN
1442,Mua Bảo Hiểm Yamaha chính hiệu kèm theo xe-giá...,180000,-,https://muaban.net/xe-may/phu-tung-xe-may/mua-...,-,-,-,-,-,-,-,-,0,NaN
1443,"Gia Công Xi Mạ Chất Lượng, Giá Hợp Lý, Bảo Hà...",5000,-,https://muaban.net/xe-may/phu-tung-xe-may/gia-...,-,-,-,-,-,-,-,-,0,NaN
1444,"AE cơ khí cần nhuộm đen, xi đen chiều dài 2m, ...",10000,-,https://muaban.net/xe-may/phu-tung-xe-may/ae-c...,-,-,-,-,-,-,-,-,0,NaN


In [19]:
all = all.append(webike_data[~webike_data.index.isin(potential_matches['level_1'])].rename(columns=webike_inv).rename(columns=inverse_dict_field))

/tmp/ipykernel_63266/3389037095.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all = all.append(webike_data[~webike_data.index.isin(potential_matches['level_1'])].rename(columns=webike_inv).rename(columns=inverse_dict_field))


In [20]:
all

,name,price,address,url,brand,registerYear,status,type,origin,capacity,color,vehicleType,km,imageUrl,Đời xe,Phân khối
0,CẦN BÁN XE MÁY HONDA VISION 110 ĐỜI 2022,15000000,"Quận 5, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-ban-x...,Honda,2022.0,Mới,Tay ga,Nước khác,101 - 175 cc,Đen,-,0,NaN,NaN,NaN
1,"Chuyên thu mua các loại xe máy cũ và mới, xe h...",0,"Quận Đống Đa, Hà Nội",https://muaban.net/xe-may/mua-xe-may/chuyen-th...,-,-,-,-,-,-,-,-,0,NaN,NaN,NaN
2,HEAD SƠN TÙNG 1 - TRỢ GIÁ MUA XE TƯNG BỪNG-VUI...,31990000,"Huyện Hoài Đức, Hà Nội",https://muaban.net/xe-may/ban-xe-may/head-son-...,Honda,-,Mới,Tay côn/Moto,-,-,-,-,0,NaN,NaN,NaN
3,Cần thanh lý lô xe honda AirBlade 150 nhập khẩu,19000000,"TP. Thủ Đức - Quận 9, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-thanh...,Honda,2022.0,Mới,Tay ga,Nước khác,101 - 175 cc,Đen,Air Blade,0,NaN,NaN,NaN
4,CẦN BÁN XE MÁY HONDA SH150 ĐỜI 2022,32000000,"Quận 5, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-ban-x...,Honda,2023.0,Mới,Tay ga,Nước khác,101 - 175 cc,Ghi/Xám,SH,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,Sprint S125 Iget Abs Màu Xanh Sport 2021,70000000.0,"Quận 5 , Hồ Chí Minh",https://www.webike.vn/cho-xe-may/bike-detail/s...,NaN,4 tháng trước,NaN,Khác,-,NaN,-,PIAGGIO khác,5 000 - 9 999 km,NaN,-,-
2996,Xe Dream 2000 - Cần bán HONDA khác,11500000.0,"Hóc Môn , Hồ Chí Minh",https://www.webike.vn/cho-xe-may/bike-detail/x...,NaN,4 tháng trước,NaN,Khác,-,NaN,-,HONDA khác,30 000 - 34 999 km,NaN,-,-
2997,raider đen nhám biển 71 chính chủ,25500000.0,"Bình Thạnh , Hồ Chí Minh",https://www.webike.vn/cho-xe-may/bike-detail/r...,NaN,4 tháng trước,NaN,Xe côn tay,-,NaN,-,SUZUKI Raider R150 2018,0 - 4 999 km,NaN,2018,150 cc
2998,Virago 125 HQCN up full Cafe Racer,19000000.0,-,https://www.webike.vn/cho-xe-may/bike-detail/v...,NaN,-,NaN,-,-,NaN,-,-,-,NaN,-,-


In [21]:
candidates = indexer.index(muaban_data, okxe_data)

In [22]:
muaban_okxe

{'price': 'price',
 'name': 'name',
 'address': 'address',
 'url': 'url',
 'Tình trạng': 'used_status',
 'Hãng xe': 'brand'}

In [23]:
# Build the indexer
indexer = recordlinkage.Index()
# Can use full or block
# indexer.full()
# indexer.block(left_on='State', right_on='Provider State')

# Use sortedneighbor as a good option if data is not clean
indexer.sortedneighbourhood(left_on='name', right_on='name')
candidates = indexer.index(muaban_data, okxe_data)
compare = recordlinkage.Compare()
# compare.exact('City', 'Provider City', label='City')
compare.string('name','name', method='levenshtein',threshold=0.8,label='Name')
compare.string('address', 'address', method='jarowinkler', threshold=0.7, label='address')
compare.string('Tình trạng', 'used_status', method='jarowinkler', threshold=0.5, label='Tình trạng')
compare.string('Hãng xe', 'brand', method='jarowinkler', threshold=0.5, label='Hãng xe')
compare.string('Dòng xe', 'model', method='jarowinkler', threshold=0.5, label='Dòng xe')
features = compare.compute(candidates, muaban_data,
                           okxe_data)
# Get the potential matches
potential_matches = features[features.sum(axis=1) > 3].reset_index()
potential_matches

,level_0,level_1,Name,address,Tình trạng,Hãng xe,Dòng xe


In [24]:
all = all.append(okxe_data[~okxe_data.index.isin(potential_matches['level_1'])].rename(columns=okxe_inv).rename(columns=inverse_dict_field))
all

/tmp/ipykernel_63266/1937918524.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all = all.append(okxe_data[~okxe_data.index.isin(potential_matches['level_1'])].rename(columns=okxe_inv).rename(columns=inverse_dict_field))


,name,price,address,url,brand,registerYear,status,type,origin,capacity,...,used_distance,Giảm xóc - Phuộc,Phanh,Động cơ,Đồng hồ,Khoá,Hệ thống đèn,Hệ thống tự động,Công nghệ kết nối,Khác
0,CẦN BÁN XE MÁY HONDA VISION 110 ĐỜI 2022,15000000,"Quận 5, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-ban-x...,Honda,2022.0,Mới,Tay ga,Nước khác,101 - 175 cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Chuyên thu mua các loại xe máy cũ và mới, xe h...",0,"Quận Đống Đa, Hà Nội",https://muaban.net/xe-may/mua-xe-may/chuyen-th...,-,-,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HEAD SƠN TÙNG 1 - TRỢ GIÁ MUA XE TƯNG BỪNG-VUI...,31990000,"Huyện Hoài Đức, Hà Nội",https://muaban.net/xe-may/ban-xe-may/head-son-...,Honda,-,Mới,Tay côn/Moto,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cần thanh lý lô xe honda AirBlade 150 nhập khẩu,19000000,"TP. Thủ Đức - Quận 9, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-thanh...,Honda,2022.0,Mới,Tay ga,Nước khác,101 - 175 cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CẦN BÁN XE MÁY HONDA SH150 ĐỜI 2022,32000000,"Quận 5, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-ban-x...,Honda,2023.0,Mới,Tay ga,Nước khác,101 - 175 cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,Honda Winner 150,17500000,Tp. Hà Nội,https://okxe.vn/products/honda-winner-150-1697565,Honda,NaN,Đã sử dụng,2,NaN,NaN,...,0-5000km,Phuộc ống lồng,"Phanh ABS,Phanh thủy lực",Phun xăng điện tử Fi / PGM-FI,-,Khóa chống trộm,-,-,-,-
328,Bajaj Pulsar 125,31000000,Tp. Hà Nội,https://okxe.vn/products/bajaj-pulsar-125-1697564,Bajaj,NaN,Đã sử dụng,2,NaN,NaN,...,0-5000km,-,-,-,-,-,-,-,-,-
329,Honda Air Blade 125 FI,22000000,Tp. Hà Nội,https://okxe.vn/products/honda-air-blade-125-f...,Honda,NaN,Đã sử dụng,1,NaN,NaN,...,0-5000km,-,"Phanh cơ,Phanh Đĩa",Phun xăng điện tử Fi / PGM-FI,-,-,-,Hệ thống ngắt động cơ tạm thời - Idling Stop,-,-
330,Honda Click 110,6500000,Tp. Hà Nội,https://okxe.vn/products/honda-click-110-1697561,Honda,NaN,Đã sử dụng,2,NaN,NaN,...,0-5000km,-,"Phanh thắng đùm/ tang trống,Phanh Đĩa",Phun xăng điện tử Fi / PGM-FI,-,-,-,Hệ thống ngắt động cơ tạm thời - Idling Stop,-,-


In [25]:
okxe_inv

{'price': 'price',
 'name': 'name',
 'address': 'address',
 'url': 'url',
 'used_status': 'Tình trạng',
 'brand': 'Hãng xe'}

In [26]:
xemayhoangkien_inv

{'price': 'price',
 'address': 'address',
 'name': 'name',
 'url': 'url',
 'Năm đăng ký': 'Năm đăng ký',
 'Màu sắc': 'Màu xe',
 'Dung tích': 'Dung tích xe máy',
 'Thương hiệu': 'Hãng xe',
 'Loại': 'Loại xe máy'}

In [27]:
# Build the indexer
indexer = recordlinkage.Index()
# Can use full or block
# indexer.full()
# indexer.block(left_on='State', right_on='Provider State')

# Use sortedneighbor as a good option if data is not clean
indexer.sortedneighbourhood(left_on='name', right_on='name')
candidates = indexer.index(muaban_data, okxe_data)
compare = recordlinkage.Compare()
# compare.exact('City', 'Provider City', label='City')
compare.string('name','name', method='levenshtein',threshold=0.8,label='Name')
compare.exact('Năm đăng ký', 'Năm đăng ký', label='Năm đăng ký')
compare.string('Màu xe', 'Màu sắc', method='levenshtein', threshold=0.5, label='Màu xe')
compare.string('Dung tích xe máy', 'Dung tích', method='jarowinkler', threshold=0.7, label='Dung tích xe máy')
compare.string('Hãng xe', 'Thương hiệu', method='jarowinkler', threshold=0.5, label='Hãng xe')
compare.string('Loại xe máy', 'Loại', method='jarowinkler', threshold=0.5, label='Loại xe máy')
features = compare.compute(candidates, muaban_data,
                           xemayhoangkien_data)
# Get the potential matches
potential_matches = features[features.sum(axis=1) > 3].reset_index()
potential_matches

,level_0,level_1,Name,Năm đăng ký,Màu xe,Dung tích xe máy,Hãng xe,Loại xe máy


In [28]:
all = all.append(xemayhoangkien_data[~xemayhoangkien_data.index.isin(potential_matches['level_1'])].rename(columns=xemayhoangkien_inv).rename(columns=inverse_dict_field))
all

/tmp/ipykernel_63266/1265139995.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all = all.append(xemayhoangkien_data[~xemayhoangkien_data.index.isin(potential_matches['level_1'])].rename(columns=xemayhoangkien_inv).rename(columns=inverse_dict_field))


,name,price,address,url,brand,registerYear,status,type,origin,capacity,...,Giảm xóc - Phuộc,Phanh,Động cơ,Đồng hồ,Khoá,Hệ thống đèn,Hệ thống tự động,Công nghệ kết nối,Khác,MSP
0,CẦN BÁN XE MÁY HONDA VISION 110 ĐỜI 2022,15000000,"Quận 5, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-ban-x...,Honda,2022.0,Mới,Tay ga,Nước khác,101 - 175 cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Chuyên thu mua các loại xe máy cũ và mới, xe h...",0,"Quận Đống Đa, Hà Nội",https://muaban.net/xe-may/mua-xe-may/chuyen-th...,-,-,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HEAD SƠN TÙNG 1 - TRỢ GIÁ MUA XE TƯNG BỪNG-VUI...,31990000,"Huyện Hoài Đức, Hà Nội",https://muaban.net/xe-may/ban-xe-may/head-son-...,Honda,-,Mới,Tay côn/Moto,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cần thanh lý lô xe honda AirBlade 150 nhập khẩu,19000000,"TP. Thủ Đức - Quận 9, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-thanh...,Honda,2022.0,Mới,Tay ga,Nước khác,101 - 175 cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CẦN BÁN XE MÁY HONDA SH150 ĐỜI 2022,32000000,"Quận 5, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-ban-x...,Honda,2023.0,Mới,Tay ga,Nước khác,101 - 175 cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,Vision 29V-058CG,16300000,52 Chùa Hà - Cầu Giấy - Hà Nội,https://xemayhoangkien.com/vision-29v-058cg,Honda,2012,NaN,Vision,NaN,110cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29V-058CG
779,Lead 29V-580BF,39900000,52 Chùa Hà - Cầu Giấy - Hà Nội,https://xemayhoangkien.com/lead-29v-580bf,Honda,2022,NaN,Lead,NaN,125cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29V-580BF
780,Winner 18B-937CF,18500000,52 Chùa Hà - Cầu Giấy - Hà Nội,https://xemayhoangkien.com/winner-18b-937cf,Honda,2017,NaN,Winner,NaN,150cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18B-937CF
781,Shmode 29L-553AG,57500000,52 Chùa Hà - Cầu Giấy - Hà Nội,https://xemayhoangkien.com/shmode-29l-553ag,Honda,2022,NaN,SH Mode,NaN,125cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Shmode 29L-553AG


In [32]:
result = all[['name', 'price', 'address', 'url', 'brand', 'registerYear', 'status', 'type', 'origin', 'capacity', 'color', 'vehicleType', 'km', 'imageUrl']]

In [33]:
result['imageUrl']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
778    NaN
779    NaN
780    NaN
781    NaN
782    NaN
Name: imageUrl, Length: 5560, dtype: object

In [30]:
# result = result.fillna('-')
# result.to_csv('final_data.csv')